In [3]:
import pickle
from sklearn import model_selection, metrics
import numpy as np
import pandas as pd

# AUDIO_EMBEDDINGS = '/Users/agnieszkalenart/Documents/mannheim/master_thesis/cmn_audio_wav2vec2.pickle'
AUDIO_EMBEDDINGS = '/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/feature_extraction/extract_wav2vec2/cmn_audio_wav2vec2.pickle'
trainID = pickle.load(open("/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/CMN_wav2vec2/IEMOCAP/data/trainID_new.pkl",'rb'), encoding="latin1")
testID = pickle.load(open("/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/CMN_wav2vec2/IEMOCAP/data/testID_new.pkl",'rb'), encoding="latin1")
valID,_ = model_selection.train_test_split(testID, test_size=.4, random_state=1227)

Preprocess data

In [5]:
audio_emb = pickle.load(open(AUDIO_EMBEDDINGS, 'rb'), encoding="latin1")
df = pd.DataFrame.from_dict(audio_emb, orient='index')
df = df.rename(columns={0: 'feature_array'})
transcripts, labels, own_historyID, other_historyID, own_historyID_rank, other_historyID_rank = pickle.load(open("/Users/agnieszkalenart/Documents/mannheim/master_thesis/thesis_erc/CMN_wav2vec2/IEMOCAP/data/dataset.pkl",'rb'), encoding="latin1")

In [3]:
audio_emb['Ses01F_impro01_F006']

array([[ 0.2677064 ,  0.22839022,  0.21444944, ..., -0.01397762,
         0.18952939, -0.05317491],
       [ 0.2020247 ,  0.21179563,  0.09720572, ...,  0.05320582,
         0.270535  , -0.194329  ],
       [ 0.05295341,  0.16415058,  0.50467575, ...,  0.18705377,
         0.3060935 ,  0.0306994 ],
       ...,
       [-0.10022137,  0.09075346, -0.07299747, ...,  0.44685835,
         0.48565978, -0.4516583 ],
       [ 0.01174384,  0.18728483,  0.01521533, ...,  0.41088742,
         0.3138377 , -0.3793756 ],
       [ 0.25195765,  0.2377499 , -0.2239635 , ...,  0.07809353,
         0.12065947, -0.360631  ]], dtype=float32)

In [6]:
max_size = 0
for i in range(len(df['feature_array'])):
    if df['feature_array'][i].shape[0] > max_size:
        max_size = df['feature_array'][i].shape[0]
max_size

749

In [7]:
# Define the desired size for the arrays
desired_shape = (max_size, df['feature_array'][0].shape[1])

# Function to pad or trim the array to the desired shape
def resize_array(arr):
    resized_arr = np.zeros(desired_shape)
    resized_arr[:arr.shape[0], :arr.shape[1]] = arr[:desired_shape[0], :desired_shape[1]]
    resized_arr = resized_arr.ravel()

    return resized_arr

df['feature_array'] = df['feature_array'].apply(resize_array)



In [8]:
# Function to extract values from the array and create separate columns
def extract_values(row):
    return pd.Series(row[0])

# Apply the extract_values function to the column of arrays
new_df = df.apply(extract_values, axis=1)

# Concatenate the original DataFrame with the new DataFrame
df = pd.concat([df, new_df], axis=1)
print(df.shape)
print(df.head())

# Drop the original column
df = df.drop('feature_array', axis=1)
print(df.shape)
print(df.head())

(1819, 575233)
                                                         feature_array   
Ses01F_impro01_F000  [0.37165045738220215, 0.3103700876235962, 0.06...  \
Ses01F_impro01_F001  [0.18848827481269836, 0.3724874258041382, 0.17...   
Ses01F_impro01_F002  [0.16338609158992767, 0.3216764032840729, -0.1...   
Ses01F_impro01_F003  [0.1896759420633316, 0.13979607820510864, 0.17...   
Ses01F_impro01_F004  [0.10581693798303604, 0.31410273909568787, 0.0...   

                            0         1         2         3         4   
Ses01F_impro01_F000  0.371650  0.310370  0.068409  0.188304  0.167854  \
Ses01F_impro01_F001  0.188488  0.372487  0.174650  0.177600  0.081944   
Ses01F_impro01_F002  0.163386  0.321676 -0.128737 -0.073644  0.049101   
Ses01F_impro01_F003  0.189676  0.139796  0.172310  0.111607  0.459044   
Ses01F_impro01_F004  0.105817  0.314103  0.002698  0.122206 -0.129063   

                            5         6         7         8  ...  575222   
Ses01F_impro01_F000 -0.10

In [8]:
df.loc['Ses01F_impro01_F006']

0         0.267706
1         0.228390
2         0.214449
3         0.172324
4         0.140954
            ...   
575227    0.000000
575228    0.000000
575229    0.000000
575230    0.000000
575231    0.000000
Name: Ses01F_impro01_F006, Length: 575232, dtype: float64

In [9]:
# filter 'xxx', 'sur', 'oth', 'dis', 'fea' from labels out
todrop_values = ['xxx', 'sur', 'oth', 'dis', 'fea']
todrop_keys = [key for key, value in labels.items() if value in todrop_values]
for key in todrop_keys:
    if key in labels:
        del labels[key]

In [10]:
labels.items()

dict_items([('Ses01F_script02_2_F042', 'hap'), ('Ses01F_script02_2_F043', 'hap'), ('Ses01F_script02_2_F040', 'sad'), ('Ses01F_script02_2_F041', 'neu'), ('Ses01F_script02_2_F046', 'sad'), ('Ses01F_script02_2_F047', 'neu'), ('Ses01F_script02_2_F044', 'hap'), ('Ses01F_script02_2_F045', 'hap'), ('Ses05F_script01_1_M010', 'fru'), ('Ses05F_script01_1_M011', 'fru'), ('Ses05F_script01_1_M012', 'fru'), ('Ses05F_script01_1_M013', 'ang'), ('Ses05F_script01_1_M014', 'fru'), ('Ses05F_script01_1_M015', 'fru'), ('Ses05F_script01_1_M016', 'sad'), ('Ses05F_script01_1_M017', 'fru'), ('Ses05M_script02_2_F030', 'sad'), ('Ses05M_script02_2_F031', 'sad'), ('Ses05M_script02_2_F032', 'sad'), ('Ses05M_script02_2_F034', 'hap'), ('Ses05M_script02_2_F037', 'hap'), ('Ses05M_script02_2_F038', 'hap'), ('Ses05F_script03_2_F041', 'ang'), ('Ses05F_script03_2_F040', 'ang'), ('Ses01M_script03_1_M028', 'exc'), ('Ses01M_script03_1_M029', 'exc'), ('Ses01M_script03_1_M021', 'hap'), ('Ses01M_script03_1_M022', 'hap'), ('Ses01M

In [10]:
# create embedding for filtered labels
dict_filtered_out = {}
for key in todrop_keys:
    if key in df.index:
        dict_filtered_out[key] = np.zeros(100)

In [12]:
dict_filtered_out

{'Ses01M_script03_1_M020': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Ses01M_script03_1_M023': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'Ses01M_script03_1_M024': array([0., 0., 0., 0.

In [11]:
# filter 'xxx', 'sur', 'oth', 'dis', 'fea' from df out
todrop_values = ['xxx', 'sur', 'oth', 'dis', 'fea']
# Find the common indexes
common_indexes = set(df.index).intersection(todrop_keys)
# Drop the specified indexes
df = df.drop(common_indexes)
for key in todrop_keys:
    if key in audio_emb:
        del audio_emb[key]

In [12]:
label_idx = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}
labels_array = np.asarray([label_idx[labels[ID]] for ID in audio_emb.keys()])

Neural Net without cross-validation

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model


# Split the data into features and labels
features = df
labels = pd.Series(labels_array)
indices = df.index

# Split the data into training, validation and testing sets
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, indices, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val, indices_train, indices_val = train_test_split(features, labels, indices, test_size=0.2, random_state=42)


# Standardize the feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Define the neural network model
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(labels.unique()), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Predict the emotions on the testing set
predict_x = model.predict(X_test)
classes_x=np.argmax(predict_x,axis=1)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, classes_x)
print(f'Accuracy: {accuracy}')


# Create a new model that outputs the activations of the last layer
representation_model = Model(inputs=model.input, outputs=model.layers[-2].output)

# Extract the representations for each example
train_representations = representation_model.predict(X_train)
test_representations = representation_model.predict(X_test)

# Create dict representations
representations = {}
representations.update({idx: rep for idx, rep in zip(indices_train, train_representations)})
representations.update({idx: rep for idx, rep in zip(indices_test, test_representations)})
representations.update(dict_filtered_out)


# Save the representations as a pickle file
with open('representations_wav2vec2.pkl', 'wb') as f:
    pickle.dump(representations, f)

Epoch 1/10


2023-06-23 23:44:16.457265: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


35/35 [==============================] - 14s 344ms/step - loss: 64.6696 - accuracy: 0.3124 - val_loss: 53.6824 - val_accuracy: 0.3891
Epoch 2/10
35/35 [==============================] - 7s 192ms/step - loss: 28.9659 - accuracy: 0.5947 - val_loss: 42.5030 - val_accuracy: 0.3818
Epoch 3/10
35/35 [==============================] - 5s 140ms/step - loss: 16.3046 - accuracy: 0.7022 - val_loss: 43.6004 - val_accuracy: 0.3927
Epoch 4/10
35/35 [==============================] - 5s 135ms/step - loss: 5.9214 - accuracy: 0.8024 - val_loss: 39.7772 - val_accuracy: 0.4182
Epoch 5/10
35/35 [==============================] - 4s 105ms/step - loss: 4.3032 - accuracy: 0.8470 - val_loss: 48.7158 - val_accuracy: 0.3891
Epoch 6/10
35/35 [==============================] - 3s 99ms/step - loss: 4.1207 - accuracy: 0.8825 - val_loss: 44.8954 - val_accuracy: 0.4582
Epoch 7/10
35/35 [==============================] - 4s 103ms/step - loss: 2.4916 - accuracy: 0.9062 - val_loss: 57.0876 - val_accuracy: 0.3745
Epoch 8

In [15]:
from tensorflow.keras.models import Model
representation_model = Model(inputs=model.input, outputs=model.layers[-2].output)
# Extract the representations for each example
train_representations = representation_model.predict(X_train)
test_representations = representation_model.predict(X_test)

# Create dict representations
representations = {}
representations.update({idx: rep for idx, rep in zip(indices_train, train_representations)})
representations.update({idx: rep for idx, rep in zip(indices_test, test_representations)})
representations.update(dict_filtered_out)


# Save the representations as a pickle file
with open('representations_wav2vec2.pkl', 'wb') as f:
    pickle.dump(representations, f)


7/7 [==============================] - 1s 34ms/step


In [19]:
if 'Ses01F_impro01_F006' in indices_test:
    print("Found 'Ses01F_impro01_F006' in the list!")
else:
    print("Did not find 'Ses01F_impro01_F006' in the list.")

Did not find 'Ses01F_impro01_F006' in the list.


In [21]:
dict_filtered_out['Ses01F_impro01_F006']

KeyError: 'Ses01F_impro01_F006'

In [22]:
dict_filtered_out

{}

Neural Net with Cross-validation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K


# Split the data into features and labels
features = df
labels = pd.Series(labels_array).reset_index(drop=True)
indices = df.index
# Split the data into training and testing sets
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, indices, test_size=0.2, random_state=42)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Standardize the feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network model
def create_model():
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(100, activation='relu'))    
    model.add(Dense(len(labels.unique()), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Perform cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
cv_scores = []
best_model = None
best_score = 0

for train_index, val_index in skf.split(X_train, y_train):
    train_features, val_features = X_train[train_index], X_train[val_index]
    train_labels, val_labels = y_train[train_index], y_train[val_index]
    
    model = create_model()
    model.fit(train_features, train_labels, epochs=10, batch_size=32, verbose=0)
    
    val_pred = model.predict(val_features)
    classes_x=np.argmax(val_pred,axis=1)
    fold_score = accuracy_score(val_labels, classes_x)

    if fold_score > best_score:
        best_score = fold_score
        best_model = model

    cv_scores.append(fold_score)

# Print the cross-validation scores
print('Cross-Validation Scores:', cv_scores)
print('Average Accuracy:', np.mean(cv_scores))


# Create a new model that outputs the activations of the last layer
representation_model = Model(inputs=best_model.input, outputs=best_model.layers[-2].output)

# Extract the representations for each example
train_representations = representation_model.predict(X_train)
test_representations = representation_model.predict(X_test)

# Create dict representations
representations = {}
representations.update({idx: rep for idx, rep in zip(indices_train, train_representations)})
representations.update({idx: rep for idx, rep in zip(indices_test, test_representations)})
representations.update(dict_filtered_out)


# Save the representations as a pickle file
with open('representations_wav2vec2.pkl', 'wb') as f:
    pickle.dump(representations, f)


# Predict the emotions on the testing set
y_pred = best_model.predict(X_test)
classes_x = np.argmax(y_pred, axis=1)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, classes_x)
print(f'Test Accuracy: {accuracy}')

3/3 [==============================] - 0s 27ms/step


: 

: 

Auto-encoder

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Split the data into features and labels
features = df
labels = pd.Series(labels_array).reset_index(drop=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Standardize the feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the dimensions of the autoencoder
input_dim = X_train.shape[1]
encoding_dim = 100

# Define the autoencoder model
input_layer = Input(shape=(input_dim,))
encoder_layer = Dense(encoding_dim, activation='relu')(input_layer)
decoder_layer = Dense(input_dim, activation='relu')(encoder_layer)
autoencoder = Model(inputs=input_layer, outputs=decoder_layer)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder model
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_test, X_test))

# Create a new model that represents the encoder part of the autoencoder
encoder = Model(inputs=input_layer, outputs=encoder_layer)

# Obtain the dense representation of the features
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

# Use the encoded features for emotion classification
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(encoding_dim,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(labels.unique()), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_encoded, y_train, epochs=10, batch_size=32, validation_data=(X_test_encoded, y_test))

# Predict the emotions on the testing set
y_pred = model.predict(X_test_encoded)
classes_x=np.argmax(y_pred,axis=1)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, classes_x)
print(f'Accuracy: {accuracy}')


Epoch 1/10
35/35 [==============================] - 0s 5ms/step - loss: 0.9084 - val_loss: 1.2601
Epoch 2/10
35/35 [==============================] - 0s 4ms/step - loss: 0.8251 - val_loss: 1.2281
Epoch 3/10
35/35 [==============================] - 0s 4ms/step - loss: 0.7925 - val_loss: 1.2126
Epoch 4/10
35/35 [==============================] - 0s 4ms/step - loss: 0.7718 - val_loss: 1.1976
Epoch 5/10
35/35 [==============================] - 0s 4ms/step - loss: 0.7522 - val_loss: 1.1857
Epoch 6/10
35/35 [==============================] - 0s 4ms/step - loss: 0.7363 - val_loss: 1.1781
Epoch 7/10
35/35 [==============================] - 0s 5ms/step - loss: 0.7234 - val_loss: 1.1730
Epoch 8/10
35/35 [==============================] - 0s 4ms/step - loss: 0.7117 - val_loss: 1.1696
Epoch 9/10
35/35 [==============================] - 0s 4ms/step - loss: 0.7024 - val_loss: 1.1655
Epoch 10/10
9/9 [==============================] - 0s 662us/step
Epoch 1/10
35/35 [==============================] - 0